In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
import tensorflow.math as tfmath
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc

2024-01-09 18:32:56.992197: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-09 18:32:57.208296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [55]:
class DNN_VAE_Model(Model):
    def __init__(self,input_dim, h_dim_1, h_dim_2, latent_dim):
        super(DNN_VAE_Model,self).__init__()
        self.e1=layers.Dense(h_dim_1, activation='relu')
        self.e2=layers.Dense(h_dim_2, activation='relu')
        self.mean=layers.Dense(latent_dim, activation='relu')
        self.logvar=layers.Dense(latent_dim, activation='relu')

        self.d1=layers.Dense(h_dim_2, activation='relu')
        self.d2=layers.Dense(h_dim_1, activation='relu')
        self.d3=layers.Dense(input_dim)
        
    def sample(self,m,logvar):
        std=tfmath.exp(0.5*logvar)
        eps=tf.random.normal(tf.shape(m))
        x=m+eps*std
        return x
    
    def call(self, inputs):
        x= self.e1(inputs)
        x= self.e2(x)
        m=self.mean(x)
        logvar=self.logvar(x)
        self.m=m
        self.lv=logvar

        x=self.sample(m,logvar)

        x=self.d1(x)
        x=self.d2(x)
        z=self.d3(x)
        return z
    
    def loss_function(self,y_true,y_pred,beta=1):
        """masked mse"""
        mask = K.cast(K.not_equal(y_true, 0), K.floatx())
        squared_difference = K.square(mask * (y_pred - y_true))
        L1= K.mean(squared_difference)

        mu=self.m
        logv=self.lv

        KL=-0.5*K.mean(1+logv-mu**2 - tfmath.exp(logv))

        loss=L1+beta*KL
        return loss





In [33]:
def sample(m,logvar):
    std=tfmath.exp(0.5*logvar)
    eps=tf.random.normal(tf.shape(m))

    x=m+eps*std
    return x
    

def create_VAE(input_dim, h_dim_1, h_dim_2, latent_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(h_dim_1, activation='relu')(inputs)
    x = layers.Dense(h_dim_2, activation='relu')(x)
    m = layers.Dense(latent_dim, activation='relu')(x)
    logvar = layers.Dense(latent_dim, activation='relu')(x)

    x = sample(m,logvar)

    x = layers.Dense(h_dim_2, activation='relu')(x)
    x = layers.Dense(h_dim_1, activation='relu')(x)
    outputs = layers.Dense(input_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [21]:
def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

In [34]:
def combined_loss_function(y_true,y_pred,beta=1):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    L1= K.mean(squared_difference)

    mu=mu_from_run
    logv=logv_from_run

    KL=-0.5*K.mean(1+logv-mu**2 - tfmath.exp(logv))

    loss=L1+beta*KL
    return loss

In [3]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

truthtable=[]

threshold=50
for i, batch in enumerate(Dataset):
  if np.sum(batch[:,0])>=threshold:
    truthtable+=[1]
  else:
    truthtable+=[0]

event_pt_br=[]
Data_Test_full=Dataset[2000001:3600000,:,:]
for j, br_1 in enumerate(Data_Test_full):
  event_pt_br+=[np.sum(br_1[:,0])]

for i, batch in enumerate(Dataset):
  pt_sum=0
  for j, particle in enumerate(Dataset[i,:,:]):
    if particle[3]!=0:
      pt_sum+=particle[0]
  for j, particle in enumerate(Dataset[i,:,:]):
    particle[0]=particle[0]/pt_sum

Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Test_Truth=truthtable[2000001:3600000]
Data_Validate=Dataset[3600001:4000000,:,0:3]

In [4]:
Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [56]:
INPUT_DIM = 57
H_DIM_1 = 32
H_DIM_2 = 16
LATENT_DIM = 8
DNN_VAE = DNN_VAE_Model(input_dim=INPUT_DIM, h_dim_1=H_DIM_1, h_dim_2=H_DIM_2, latent_dim=LATENT_DIM)
DNN_VAE.build((1,57))
DNN_VAE.summary()

Model: "dnn_vae__model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_100 (Dense)           multiple                  1856      
                                                                 
 dense_101 (Dense)           multiple                  528       
                                                                 
 dense_102 (Dense)           multiple                  136       
                                                                 
 dense_103 (Dense)           multiple                  136       
                                                                 
 dense_104 (Dense)           multiple                  144       
                                                                 
 dense_105 (Dense)           multiple                  544       
                                                                 
 dense_106 (Dense)           multiple            

In [57]:
DNN_VAE.compile(optimizer='adam', loss=DNN_VAE.loss_function)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 50
BATCH_SIZE = 1024

history = DNN_VAE.fit(x=Data_Train_Flat, y=Data_Train_Flat, validation_data=(Data_Val_Flat,Data_Val_Flat), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

Epoch 1/50
1954/1954 [==============================] - 13s 5ms/step - loss: 0.2165 - val_loss: 0.2154 - lr: 0.0010
Epoch 2/50
1954/1954 [==============================] - 10s 5ms/step - loss: 0.2156 - val_loss: 0.2153 - lr: 0.0010
Epoch 3/50
1954/1954 [==============================] - 11s 6ms/step - loss: 0.2156 - val_loss: 0.2153 - lr: 0.0010
Epoch 4/50
1954/1954 [==============================] - 10s 5ms/step - loss: 0.2155 - val_loss: 0.2153 - lr: 0.0010
Epoch 5/50
1952/1954 [============================>.] - ETA: 0s - loss: 0.2156
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1954/1954 [==============================] - 10s 5ms/step - loss: 0.2155 - val_loss: 0.2153 - lr: 0.0010
Epoch 6/50
1954/1954 [==============================] - 10s 5ms/step - loss: 0.2155 - val_loss: 0.2153 - lr: 1.0000e-04
Epoch 7/50
1954/1954 [==============================] - 10s 5ms/step - loss: 0.2155 - val_loss: 0.2153 - lr: 1.0000e-04
Epoch 8/50
1954/1954 [=============

In [ ]:
DNN_VAE.save('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/40MHZ_norm_DNN_VAE.keras')